# Практическое задание по занятию Denosing Diffusional Models

## Задание

0) Скачайте репозиторий git clone https://github.com/awjuliani/pytorch-diffusion.git

1) Обучите модель для датасета Fasion MNIST. Продемонстрируйте обратный диффузионный процесс с нескольких random seeds (1 балл)

2) Добавьте к обучению DDPM условие на метку с помощью Classifier Free Guidance https://arxiv.org/abs/2207.12598. (1 балл)

3) Обучите модель в режиме inpainting. Параграф 4.1 в https://arxiv.org/pdf/2201.09865.pdf . (1 балл)


In [1]:
!git clone https://github.com/awjuliani/pytorch-diffusion.git
!cp ./pytorch-diffusion/data.py data.py
!cp ./pytorch-diffusion/model.py model.py
!cp ./pytorch-diffusion/modules.py modules.py
!cp ./pytorch-diffusion/entry.ipynb entry.ipynb

!pip install pytorch-lightning


Cloning into 'pytorch-diffusion'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 35 (delta 15), reused 29 (delta 10), pack-reused 0
Unpacking objects: 100% (35/35), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 34.6 MB/s 
     |████████████████████████████████| 512 kB 73.2 MB/s 
     |████████████████████████████████| 125 kB 78.3 MB/s 


In [7]:
from data import DiffSet
from model import DiffusionModel

import torch
from torch.utils.data import DataLoader

import pytorch_lightning as pl

import imageio

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.device(device)

device(type='cuda')

In [8]:
train_dataset = DiffSet(True, "Fashion")
val_dataset = DiffSet(False, "Fashion")

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)

In [9]:
model = DiffusionModel(train_dataset.size*train_dataset.size, 1000, train_dataset.depth).to(device)

In [10]:
trainer = pl.Trainer(
    max_epochs=10, 
    log_every_n_steps=10, 
    gpus=1, 
    auto_select_gpus=True,
)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.trainer.connectors.accelerator_connector:Auto select gpus: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(model, train_loader, val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name  | Type       | Params
--------------------------------------
0  | inc   | DoubleConv | 37.7 K
1  | down1 | Down       | 295 K 
2  | down2 | Down       | 1.2 M 
3  | down3 | Down       | 2.4 M 
4  | up1   | Up         | 6.2 M 
5  | up2   | Up         | 1.5 M 
6  | up3   | Up         | 406 K 
7  | outc  | OutConv    | 65    
8  | sa1   | SAWrapper  | 395 K 
9  | sa2   | SAWrapper  | 395 K 
10 | sa3   | SAWrapper  | 99.6 K
--------------------------------------
12.9 M    Trainable params
0         Non-trainable params
12.9 M    Total params
51.676    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [12]:
def generate_image(gif_shape=[1, 1], n_hold_final=10, random_seed=42):
    torch.manual_seed(random_seed)
    sample_batch_size = gif_shape[0] * gif_shape[1]

    gen_samples = []
    x = torch.randn((sample_batch_size, train_dataset.depth, train_dataset.size, train_dataset.size))
    sample_steps = torch.arange(model.t_range-1, 0, -1)
    for t in sample_steps:
        x = model.denoise_sample(x, t)
        if t % 50 == 0:
            gen_samples.append(x)
    for _ in range(n_hold_final):
        gen_samples.append(x)
    gen_samples = torch.stack(gen_samples, dim=0).moveaxis(2, 4).squeeze(-1)
    return (gen_samples.clamp(-1, 1) + 1) / 2

def create_gif(gen_samples, gif_shape=[1, 1], random_seed=42):
    gen_samples = (gen_samples * 255).type(torch.uint8)
    gen_samples = gen_samples.reshape(-1, gif_shape[0], gif_shape[1], train_dataset.size, train_dataset.size, train_dataset.depth)

    def stack_samples(gen_samples, stack_dim):
        gen_samples = list(torch.split(gen_samples, 1, dim=1))
        for i in range(len(gen_samples)):
            gen_samples[i] = gen_samples[i].squeeze(1)
        return torch.cat(gen_samples, dim=stack_dim)

    gen_samples = stack_samples(gen_samples, 2)
    gen_samples = stack_samples(gen_samples, 2)

    imageio.mimsave(
        f"pred_seed_{random_seed}.gif",
        list(gen_samples),
        fps=5,
    )



In [13]:
for seed in [42, 101, 37]:
    create_gif(generate_image(random_seed=seed), random_seed=seed)

In [14]:
from my_model import ClassifierFreeGuidance
from my_data import DiffSetWithClass

In [15]:
train_dataset = DiffSetWithClass(True, "Fashion")
val_dataset = DiffSetWithClass(False, "Fashion")

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True)

In [17]:
num_classes = 10 # классы в фешн мнисте
embed_size = 4 # размерность эмбеддинга класса

model = ClassifierFreeGuidance(
    num_classes, 
    embed_size, 
    train_dataset.size*train_dataset.size, 
    1000, 
    train_dataset.depth
).to(device)

trainer = pl.Trainer(
    max_epochs=20, 
    log_every_n_steps=10, 
    gpus=1, 
    auto_select_gpus=True,
)


INFO:pytorch_lightning.trainer.connectors.accelerator_connector:Auto select gpus: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(model, train_loader, val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name             | Type       | Params
-------------------------------------------------
0  | linear_map_class | Sequential | 64    
1  | inc              | DoubleConv | 40.0 K
2  | down1            | Down       | 295 K 
3  | down2            | Down       | 1.2 M 
4  | down3            | Down       | 2.4 M 
5  | up1              | Up         | 6.2 M 
6  | up2              | Up         | 1.5 M 
7  | up3              | Up         | 406 K 
8  | outc             | OutConv    | 65    
9  | sa1              | SAWrapper  | 395 K 
10 | sa2              | SAWrapper  | 395 K 
11 | sa3              | SAWrapper  | 99.6 K
-------------------------------------------------
12.9 M    Trainable params
0         Non-trainable params
12.9 M    Total params
51.686    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [19]:
def generate_image(gif_shape=[1, 1], n_hold_final=10, random_seed=42, target=0):
    torch.manual_seed(random_seed)
    sample_batch_size = gif_shape[0] * gif_shape[1]

    x = torch.randn((sample_batch_size, train_dataset.depth, train_dataset.size, train_dataset.size))

    # теперь у нас есть желаемый target
    c = torch.tensor(target).unsqueeze(0)
    # переводим его в one_hot
    c = torch.nn.functional.one_hot(c, model.num_classes).float()
    # берём его эмбеддинг
    with torch.no_grad():
        emb_c = model.linear_map_class(c)
    # также подгоняем шейп чтобы эмбеддинг попал в канал
    emb_c = emb_c.view(*emb_c.shape, 1, 1)
    emb_c = emb_c.expand(-1, -1, x.shape[-2], x.shape[-1])


    gen_samples = []
    sample_steps = torch.arange(model.t_range-1, 0, -1)
    for t in sample_steps:
        x = model.denoise_sample(x, t, emb_c, 0.2)
        if t % 50 == 0:
            gen_samples.append(x)
    for _ in range(n_hold_final):
        gen_samples.append(x)
    gen_samples = torch.stack(gen_samples, dim=0).moveaxis(2, 4).squeeze(-1)
    return (gen_samples.clamp(-1, 1) + 1) / 2

def create_gif(gen_samples, gif_shape=[1, 1], random_seed=42, target=0):
    gen_samples = (gen_samples * 255).type(torch.uint8)
    gen_samples = gen_samples.reshape(-1, gif_shape[0], gif_shape[1], train_dataset.size, train_dataset.size, train_dataset.depth)

    def stack_samples(gen_samples, stack_dim):
        gen_samples = list(torch.split(gen_samples, 1, dim=1))
        for i in range(len(gen_samples)):
            gen_samples[i] = gen_samples[i].squeeze(1)
        return torch.cat(gen_samples, dim=stack_dim)

    gen_samples = stack_samples(gen_samples, 2)
    gen_samples = stack_samples(gen_samples, 2)

    imageio.mimsave(
        f"pred_seed_{random_seed}_class{target}.gif",
        list(gen_samples),
        fps=5,
    )

In [24]:
for target in [0, 4, 9]:
    # все с рандом сидом 42
    create_gif(generate_image(target=target), target=target)

for seed in [220, 7721, 9090]:
    # класс 6 с разными сидами
    create_gif(generate_image(random_seed=seed, target=6), random_seed=seed, target=6)


Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: